In [98]:
from dimod import ConstrainedQuadraticModel, Real, quicksum

def mps2cmq(filename):
    ### read the mps file ###
    mpsFile = open(filename, "r")
    mpsLine = mpsFile.readlines()
    mpsLine = [s.replace('\n', '') for s in mpsLine]

    readRow, readCol, readRHS, readBnd, readObj = False, False, False, False, False
    equations, variables, objEq = {}, set(), None
    LHSeq, RHSeq, upBound, lowBound = {}, {}, {}, {}

    for i in range(len(mpsLine)):
        string = mpsLine[i]  
        if  string == 'ROWS':
            readRow = True
        elif string == 'COLUMNS':
            readRow = False
            readCol = True
            LHSeq = {k: [] for k in equations.keys()}
            RHSeq = {k: [] for k in equations.keys()}
        elif string == 'RHS':
            readCol = False
            readRHS = True
        elif string == 'BOUNDS':
            readRHS = False
            readBnd = True

        if readRow and string != 'ROWS':
            relation, eq = string.split()
            equations[eq] = relation
            if not readObj and relation == 'N': ObjEq = eq
        
        if readCol and string != 'COLUMNS':
            strList = string.split()
            var = strList[0]
            variables.add(var)
            for i in range(0, len(strList)//2):
                LHSeq[strList[i*2+1]].append((var, strList[i*2+2]))
        
        if readRHS and string != 'RHS':
            strList = string.split()
            for i in range(0, len(strList)//2):
                RHSeq[strList[i*2+1]] = strList[i*2+2]
        
        if readBnd and string != 'BOUNDS':
            strList = string.split()
            if strList[0] == 'UP':
                upBound[strList[2]] = strList[3]
            elif strList[0] == 'LO':
                lowBound[strList[2]] = strList[3]

    mpsFile.close()

    ### convert into cqm ###
    cqm = ConstrainedQuadraticModel()

    # variables
    x = {v: Real(v) for v in variables}

    # the objective function and constraint equations
    for eq in equations:
        currLHS = quicksum(x[v]*float(coef) for v, coef in LHSeq[eq])

        if eq == ObjEq:
            cqm.set_objective(currLHS)
        
        if equations[eq] == 'E':
            cqm.add_constraint(currLHS == float(RHSeq[eq]), label=eq)
        elif equations[eq] == 'G':
            cqm.add_constraint(currLHS >= float(RHSeq[eq]), label=eq)
        elif equations[eq] == 'L':
            cqm.add_constraint(currLHS <= float(RHSeq[eq]), label=eq)

    # constraints on the variables
    for k in upBound.keys():
        cqm.set_upper_bound(k, float(upBound[k]))
    for k in lowBound.keys():
        cqm.set_lower_bound(k, float(lowBound[k]))
    
    return cqm

In [100]:
from dwave.system import LeapHybridCQMSampler

cqm1 = mps2cmq('testprob.mps')

sampler = LeapHybridCQMSampler()

sampleset = sampler.sample_cqm(cqm1, label='Example')
feasible_sampleset = sampleset.filter(lambda d: d.is_feasible)

In [101]:
feasible_sampleset

SampleSet(rec.array([([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [ True,  True,  True],  True),
           ([ 4., -1.,  6.], 54., 1, [